In [ ]:
#run once to install the necessary libraries
#this code also needs numpy and pandas 
#https://pypi.org/project/opencv-python/
#https://github.com/duckietown/lib-dt-apriltags
#!pip install dt-apriltags
#!pip install opencv-python
#!pip install pandas

### Camera Calibration

the following section will extract camera calibration parameters from an image or video.


To use this calibration routine, you will need to print a chessboard with 20mm x 20mm squares and place in frame


Ideally, the chessboard will be on same plane as apriltags.

If using a still image, take camera image WITH SAME SETTINGS as video and name it calibrate.jpg or pass in filename as arg to calibrate()


If using a video, name video "calibrate.mp4" or pass filename into calibrate_from_video()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import dt_apriltags as ap
from math import cos,sin,sqrt,atan2
from collections import defaultdict
import csv

from calibrate import calibrate, calibrate_from_video
from track_tags import track_tags

calibration_file = "test.webm"




# camera_params: (fx, fy, cx, cy) in tuple
# mtx: camera matrix
# dist: distortion coefficients
# meters_per_px: workspace meters per camera pixel, assumes grid squares are 2cm square
camera_params, mtx, dist, meters_per_pix = calibrate(calibration_file)
camera_params, mtx, dist, meters_per_pix = calibrate_from_video(calibration_file)

(fx, fy, cx, cy) = camera_params
   
#input video is 24 fps. interval 1 means record the position of the arena per frame. 
#if you want to extract the position per second, change interval to be 24
#lower the value if you want to record more precise movements
interval=1

### Run tracking code

The following code block will create the detector and loop through the video. It will open a preview video so you can manually inspect the quality of the tracking.

All data is written to two videos and a CSV file, even if no tags are detected or some are dropped. You will have to manually process your data to handle dropped tags, etc.

Change input/output filenames, and tag type/size as necessary.

The code will probably throw some errors, that's ok as long as it tracks the tags most of the time. This block will also print a progress measure.

In [ ]:
# change detector parameters here

input_file='test.webm'
output_file='outpy.mp4'

TAG_TYPE = 'tag36h11'
tag_size = 0.028

detector = ap.Detector(families=TAG_TYPE,
                            nthreads=1,
                            quad_decimate=1.0,
                            quad_sigma=0.0,
                            refine_edges=1,
                            decode_sharpening=0.25,
                            debug=0)


# loops through video, undistorts frames, finds tags, logs, writes videos, shows preview
data = track_tags(detector,
           camera_params,
           meters_per_pix,
           mtx,
           dist,
           tag_size,
           interval,
           input_file,
           output_file)